<a href="https://colab.research.google.com/github/NawidT/gpu_programming/blob/main/learnings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pre-requisite Steps

In [7]:
!pip install opencv-python-headless

Ensure the following writefile steps work in Google Colab

In [ ]:
%%writefile code.h

int multiply(int a, int b);

Writing code.h


In [ ]:
%%writefile code.cpp

#include "code.h"
using namespace std;

int multiply(int a, int b) {
  return a * b;
};

Writing code.cpp


In [ ]:
%%writefile main.cpp

#include <iostream>
using namespace std
#include "code.h"

int main() {
  cout << multiply(6,7) << '\n';
  return 0;
};

Writing main.cpp


In [22]:
!nvidia-smi

Mon Feb 17 20:47:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [23]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


## Vector Mathematics Optimization Examples

In [25]:
%%writefile test.cu

#include <cstdio>
#define cudaCheckError() { cudaError_t e=cudaGetLastError(); if(e!=cudaSuccess) printf("CUDA error %s\n", cudaGetErrorString(e)); }

__global__ void helloCUDA() {
    printf("Hello from CUDA kernel cuda!\n");
}

int main() {
    helloCUDA<<<1,1>>>();
    cudaDeviceSynchronize();
    cudaCheckError();
    return 0;
}

Overwriting test.cu


In [27]:
!nvcc -arch=sm_75 test.cu -o test_cuda
!./test_cuda

Hello from CUDA kernel cuda!


In [1]:
%%writefile streaming.cu


#include <cuda_runtime.h>
#include <stdio.h>

__global__ void vectorAdd(const float *A, const float *B, float *C, int numElems) {
    // conditional in place when num threads exceed needed operations.
    // E.g 256 threads/block * 4 blocks = 1024 threads, but only 1000 needed
    // Extra 24 threads will be ignored
    if (threadIdx.x >= numElems) {
        return;
    }
    for (int i = 0; i < numElems; i++) {
      C[i] = A[i] + B[i];
    }
}

int main() {
    // PROBLEM: optimize a large vector addition process using streaming and paralellism

    // currently in realms of host (CPU) so can use non-cuda operations
    int numRows = 50000;
    size_t size = numRows * sizeof(float);
    float *h_A, *h_B, *h_C;

    // Allocate host memory
    h_A = (float *) malloc(size);
    h_B = (float *) malloc(size);
    h_C = (float *) malloc(size);

    // Initialize host arrays
    for (int i = 0; i < numRows; ++i) {
        h_A[i] = rand() / (float)RAND_MAX;
        h_B[i] = rand() / (float)RAND_MAX;
    }

    // Allocate memory for device arrays
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);

    // create streams to async-ly copy data to device
    cudaStream_t stream1, stream2;

    cudaStreamCreate(&stream1);
    cudaStreamCreate(&stream2);

    cudaMemcpyAsync(d_A, h_A, size, cudaMemcpyHostToDevice, stream1);
    cudaMemcpyAsync(d_B, h_B, size, cudaMemcpyHostToDevice, stream2);

    // Launch kernel<<<blocksPerGrid, threadsPerBlock, 0, stream1>>>(d_A, d_B, d_C, numRows);
    int threadsPerBlock = 256;
    int blocksPerGrid = (numRows + threadsPerBlock - 1) / threadsPerBlock;
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(h_A, h_B, h_C, size);


    // copy data back to host
    cudaMemcpyAsync(h_C, d_C, size, cudaMemcpyDeviceToHost, stream1);

    // clean up
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    free(h_A);
    free(h_B);
    free(h_C);

};



Writing streaming.cu


In [2]:
!nvcc -arch=sm_75 streaming.cu -o stream
!./stream

## Computer Vision Optimization Examples

# Applying Filters

In [20]:
%%writefile image_cv_ops.cu

#include <cuda_runtime.h>
#include <stdio.h>
#include <fstream>
#include <opencv2/opencv.hpp>
#include <iostream>
#include <cstdlib>

__global__ void apply_gaussian_blur(const float *image, float *out_image, int height, int width) {
    // conditional in place when num threads exceed needed operations.

    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= width || y >= height) {
      return;
    }

    int pixel_index = y * width + x;

    // create the gaussian kernel
    float kernel[3][3] = {
        {1.0/16, 2.0/16, 1.0/16},
        {2.0/16, 4.0/16, 2.0/16},
        {1.0/16, 2.0/16, 1.0/16}
    };

    // apply blur to image, pasting to out_image
    int half_kernel_size = 1;
    float sum = 0.0f;

    for (int i = -half_kernel_size; i <= half_kernel_size; i++) {
        for (int j = -half_kernel_size; j <= half_kernel_size; j++) {
            int neighbor_x = x + j;
            int neighbor_y = y + i;
            // Check for image edge conditions
            if (neighbor_x >= 0 && neighbor_x < width && neighbor_y >= 0 && neighbor_y < height) {
                int neighbor_index = neighbor_y * width + neighbor_x;
                sum += image[neighbor_index] * kernel[i + half_kernel_size][j + half_kernel_size];
            }
        }
    }
    out_image[pixel_index] = sum;
}

void run_gaussian_blur(const float *image, float *out_image, int height, int width) {
    // copy image to VRAM (shared memory)
    int num_pixels = height * width;
    size_t size_pixels = num_pixels * sizeof(float);
    float* d_image, *d_out_image;
    cudaMalloc(&d_image, size_pixels);
    cudaMalloc(&d_out_image, size_pixels);
    cudaMemcpy(d_image, image, size_pixels, cudaMemcpyHostToDevice);

    // TODO: apply gaussian blur to image

    dim3 block(32, 32);
    dim3 grid((width + block.x - 1) / block.x, (height + block.y - 1) / block.y);

    apply_gaussian_blur<<<grid, block>>>(d_image, d_out_image, height, width);

    // copy image back to host
    cudaMemcpy(out_image, d_out_image, size_pixels, cudaMemcpyDeviceToHost);

    // clean up
    cudaFree(d_image);
    cudaFree(d_out_image);
}

int main() {
   // open image using cv::imread()
   cv::Mat image = cv::imread("1b_cat.bmp", cv::IMREAD_GRAYSCALE);
   if (image.empty()) {
       printf("Could not open or find the image\n");
       return -1;
   }


   int num_pixels = image.rows * image.cols;
   float *h_image = (float*) malloc(num_pixels * sizeof(float));;
   float *h_out_image = (float*) malloc(num_pixels * sizeof(float));

   // Convert the image data from uchar to float.
   for (int i = 0; i < num_pixels; ++i) {
      h_image[i] = static_cast<float>(image.data[i]);
    }

   printf("number of pixels are %d \n", num_pixels);

   // apply gaussian blur to image
   run_gaussian_blur(h_image, h_out_image, image.rows, image.cols);

   // convert the output image back to cv::Mat
   cv::Mat out_image_mat(image.size(), CV_32F, h_out_image);
   cv::Mat out_image_8U;
   out_image_mat.convertTo(out_image_8U, CV_8U);
   cv::imwrite("blurred_image.jpg", out_image_8U);

   // clean up
   free(h_out_image);
   return 0;
}

Overwriting image_cv_ops.cu


In [21]:
!nvcc -arch=sm_75 image_cv_ops.cu -I/usr/include/opencv4 -L/usr/lib -lopencv_core -lopencv_imgcodecs -lopencv_highgui -lopencv_imgproc -o cv_ops
!./cv_ops

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is